In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, LargeBinary, MetaData
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.orm import sessionmaker
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from datetime import datetime
import matplotlib.pyplot as plt

load_dotenv()

True

In [2]:
db_url = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/postgres"
engine = create_engine(db_url, pool_timeout=None, connect_args={"options": "-c statement_timeout=0"})

In [3]:
sql_query = """
    SELECT
        time AS timestamp, DT.olr, speed, speeduncapped, freeflow, jamfactor, traversability, day_of_week, is_peak_hour, time_to_traverse, congestion_level, is_anomaly, cluster
    FROM diu.traffic_data AS DT
    JOIN diu.traffic_patters_cluster AS TPC
        ON DT.olr = TPC.olr
    WHERE day_of_week IS NOT NULL AND name IS NOT NULL AND method = 'DBSCAN'
    ORDER BY time DESC;
"""
traffic_data = pd.read_sql(sql_query, engine)

In [4]:
traffic_data['timestamp'] = pd.to_datetime(traffic_data['timestamp'])
traffic_data.set_index('timestamp', inplace=True)
traffic_data

,olr,speed,speeduncapped,freeflow,jamfactor,traversability,day_of_week,is_peak_hour,time_to_traverse,congestion_level,is_anomaly,cluster
timestamp,,,,,,,,,,,,
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYQbyT+4QAJBQQBAhAACgUEAYRSAAEJAfsACQ...,18.888890,19.722223,18.611110,0.0,open,0,True,31.447057,Low,False,-1
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYe5iT54wAJBQQBAj0ACgUEAYhnAAX0/p0ACQ...,24.722223,24.722223,24.722223,0.0,open,0,True,44.292133,Low,False,2
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYXkiTxsQAJBQQAASEACgUEALcaABT0EnkACQ...,31.111113,31.666668,28.611113,0.0,open,0,True,227.282129,Low,False,0
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYP8iT9igAJBQQBAhoACgUEAYVEAABpAgkACQ...,6.111111,6.111111,8.888889,2.3,open,0,True,116.018184,Moderate,True,2
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYmfiUNdAAJBQQBA60ACgUEAa9yAO3Q7kEACQ...,16.111110,16.111110,17.777779,0.9,open,0,True,379.924164,Low,False,3
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-18 16:12:21.328000+00:00,CD0BEAA5OAYTOyUAZQAJBQQCAwQACgUEAphOAAl0BOsACQ...,6.388889,6.388889,11.944445,8.1,open,3,False,873.704333,Moderate,True,1
2024-12-18 16:12:21.328000+00:00,CCkBEAAlJAYpMiT13QAJBQQBAtUACgUEAaAgAOzJCBsACQ...,NaN,NaN,20.833334,10.0,closed,3,False,NaN,Low,False,-1
2024-12-18 16:12:21.328000+00:00,CCkBEAAlJAYWFiT7JwAJBQQBAs0ACgUEAZYeAPLSBR4ACQ...,6.944445,6.944445,11.111112,3.5,open,3,False,411.263985,Moderate,True,2


In [5]:
tmp_df = traffic_data.copy()
value_counts = tmp_df['olr'].value_counts()
tmp_df = tmp_df[tmp_df['olr'].isin(value_counts[value_counts >= 100].index)]
tmp_df['is_peak_hour'] = tmp_df['is_peak_hour'].astype(int)
tmp_df['is_anomaly'] = tmp_df['is_anomaly'].astype(int)
tmp_df['congestion_level'] = tmp_df['congestion_level'].map({'High': 2, 'Moderate': 1, 'Low': 0})
tmp_df['traversability'] = tmp_df['traversability'].map({'closed': 1, 'open': 0})
tmp_df.loc[tmp_df['traversability'] == 1, 'speed'] = tmp_df.loc[tmp_df['traversability'] == 1, 'speed'].fillna(0)
tmp_df.loc[tmp_df['traversability'] == 1, 'speeduncapped'] = tmp_df.loc[tmp_df['traversability'] == 1, 'speeduncapped'].fillna(0)
tmp_df.loc[tmp_df['traversability'] == 1, 'time_to_traverse'] = tmp_df.loc[tmp_df['traversability'] == 1, 'time_to_traverse'].fillna(0)
unique_olrs = tmp_df['olr'].unique()
olr_to_int = {olr: i for i, olr in enumerate(unique_olrs)}
tmp_df['olr_hashed'] = tmp_df['olr'].map(olr_to_int)
tmp_df = tmp_df.dropna()

tmp_df

,olr,speed,speeduncapped,freeflow,jamfactor,traversability,day_of_week,is_peak_hour,time_to_traverse,congestion_level,is_anomaly,cluster,olr_hashed
timestamp,,,,,,,,,,,,,
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYQbyT+4QAJBQQBAhAACgUEAYRSAAEJAfsACQ...,18.888890,19.722223,18.611110,0.0,0,0,1,31.447057,0,0,-1,0
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYe5iT54wAJBQQBAj0ACgUEAYhnAAX0/p0ACQ...,24.722223,24.722223,24.722223,0.0,0,0,1,44.292133,0,0,2,1
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYXkiTxsQAJBQQAASEACgUEALcaABT0EnkACQ...,31.111113,31.666668,28.611113,0.0,0,0,1,227.282129,0,0,0,2
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYP8iT9igAJBQQBAhoACgUEAYVEAABpAgkACQ...,6.111111,6.111111,8.888889,2.3,0,0,1,116.018184,1,1,2,3
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYmfiUNdAAJBQQBA60ACgUEAa9yAO3Q7kEACQ...,16.111110,16.111110,17.777779,0.9,0,0,1,379.924164,0,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-18 16:12:21.328000+00:00,CD0BEAA5OAYTOyUAZQAJBQQCAwQACgUEAphOAAl0BOsACQ...,6.388889,6.388889,11.944445,8.1,0,3,0,873.704333,1,1,1,12
2024-12-18 16:12:21.328000+00:00,CCkBEAAlJAYpMiT13QAJBQQBAtUACgUEAaAgAOzJCBsACQ...,0.000000,0.000000,20.833334,10.0,1,3,0,0.000000,0,0,-1,10
2024-12-18 16:12:21.328000+00:00,CCkBEAAlJAYWFiT7JwAJBQQBAs0ACgUEAZYeAPLSBR4ACQ...,6.944445,6.944445,11.111112,3.5,0,3,0,411.263985,1,1,2,6


In [6]:
scaler = MinMaxScaler()
tmp_df[['speeduncapped', 'jamfactor', 'freeflow', 'traversability', 'day_of_week', 'time_to_traverse', 'is_peak_hour']] = scaler.fit_transform(tmp_df[['speeduncapped', 'jamfactor', 'freeflow', 'traversability', 'day_of_week', 'time_to_traverse', 'is_peak_hour']])

tmp_df

,olr,speed,speeduncapped,freeflow,jamfactor,traversability,day_of_week,is_peak_hour,time_to_traverse,congestion_level,is_anomaly,cluster,olr_hashed
timestamp,,,,,,,,,,,,,
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYQbyT+4QAJBQQBAhAACgUEAYRSAAEJAfsACQ...,18.888890,0.417647,0.558139,0.00,0.0,0.0,1.0,0.003706,0,0,-1,0
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYe5iT54wAJBQQBAj0ACgUEAYhnAAX0/p0ACQ...,24.722223,0.523529,0.813953,0.00,0.0,0.0,1.0,0.005220,0,0,2,1
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYXkiTxsQAJBQQAASEACgUEALcaABT0EnkACQ...,31.111113,0.670588,0.976744,0.00,0.0,0.0,1.0,0.026786,0,0,0,2
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYP8iT9igAJBQQBAhoACgUEAYVEAABpAgkACQ...,6.111111,0.129412,0.151163,0.23,0.0,0.0,1.0,0.013673,1,1,2,3
2025-01-26 17:20:09.782000+00:00,CCkBEAAlJAYmfiUNdAAJBQQBA60ACgUEAa9yAO3Q7kEACQ...,16.111110,0.341176,0.523256,0.09,0.0,0.0,1.0,0.044775,0,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-18 16:12:21.328000+00:00,CD0BEAA5OAYTOyUAZQAJBQQCAwQACgUEAphOAAl0BOsACQ...,6.388889,0.135294,0.279070,0.81,0.0,0.5,0.0,0.102968,1,1,1,12
2024-12-18 16:12:21.328000+00:00,CCkBEAAlJAYpMiT13QAJBQQBAtUACgUEAaAgAOzJCBsACQ...,0.000000,0.000000,0.651163,1.00,1.0,0.5,0.0,0.000000,0,0,-1,10
2024-12-18 16:12:21.328000+00:00,CCkBEAAlJAYWFiT7JwAJBQQBAs0ACgUEAZYeAPLSBR4ACQ...,6.944445,0.147059,0.244186,0.35,0.0,0.5,0.0,0.048468,1,1,2,6


In [ ]:
features = ['speeduncapped', 'jamfactor', 'freeflow', 'traversability', 'time_to_traverse', 'is_peak_hour']
Session = sessionmaker(bind=engine)
metadata = MetaData()

models_table = Table(
    'models_arima', metadata,
    Column('version', Integer, nullable=False),
    Column('created_at', String, nullable=False),
    Column('feature', String, nullable=False),
    Column('test_data', JSONB, nullable=False),
    Column('pred_data', JSONB, nullable=False),
    Column('p', Integer, nullable=False),
    Column('d', Integer, nullable=False),
    Column('q', Integer, nullable=False),
    schema='diu',
    autoload_with=engine  # Automatisch Struktur laden
)

def train_arima(series, cluster, feature, test_data, s):
    
    if len(series[feature].unique()) > 1:
        tmp = adfuller(series[feature])
        result = tmp[1] > 0.05
    else:
        result = False
    model = auto_arima(series[feature],
                       start_p=0, start_q=0,
                       max_p=5, max_q=5,
                       seasonal=True,
                       m=720,
                       stationary=result,
                       trace=True,
                       stepwise=True,
                       error_action='ignore',
                       suppress_warnings=True)
    pred = model.predict(n_periods=10)

    p, d, q = model.order
    insert_query = models_table.insert().values(
        version=2,
        created_at=datetime.now().isoformat(),
        cluster=cluster,
        feature=feature,
        test_data=test_data[feature].tolist()[:10] if feature in test_data else None,
        pred_data=pred.tolist(),
        p=p,
        d=d,
        q=q
    )
    s.execute(insert_query)
    s.commit()
    s.close()

for x in range(-1, 4):
    df = tmp_df[tmp_df['cluster'] == x]
    df = df[features]
    df = df.groupby(df.index).median()
    df.index = df.index.round('T')
    df = df[~df.index.duplicated(keep='first')]
    df = df.asfreq('2T')
    df = df.interpolate()
    
    train_size = int(len(df) * 0.8)
    train_data = df[:train_size]
    test_data = df[train_size:]
    
    for f in features:
        session = Session()
        train_arima(df, x, f, train_data, session)
        print(f'Model created: Cluster {x}, Feature {f}')

Performing stepwise search to minimize aic
